In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from math import radians, cos, sin, asin, sqrt
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="myGeocoder",timeout=20)

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

Read files

In [3]:
# Get the paths where all the files are stored
current_path = os.getcwd()
train_path = current_path + '/train.csv'
test_path = current_path + '/test.csv'

In [4]:
# read csv files
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print('shape of train data:',train_df.shape)
print('shape of test data:',test_df.shape)

shape of train data: (16756, 8)
shape of test data: (12212, 6)


In [5]:
train_df.head()

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In [6]:
test_df['Date'].value_counts()

2020-03-13    284
2020-04-22    284
2020-04-08    284
2020-04-04    284
2020-03-31    284
2020-03-17    284
2020-04-11    284
2020-04-02    284
2020-04-15    284
2020-04-13    284
2020-03-19    284
2020-03-28    284
2020-04-03    284
2020-04-20    284
2020-03-12    284
2020-03-21    284
2020-04-14    284
2020-03-22    284
2020-03-24    284
2020-04-18    284
2020-03-20    284
2020-04-06    284
2020-04-21    284
2020-04-10    284
2020-04-09    284
2020-03-18    284
2020-04-01    284
2020-03-29    284
2020-03-26    284
2020-04-07    284
2020-03-14    284
2020-04-16    284
2020-03-27    284
2020-03-15    284
2020-04-17    284
2020-03-16    284
2020-04-19    284
2020-04-05    284
2020-03-30    284
2020-04-12    284
2020-03-23    284
2020-03-25    284
2020-04-23    284
Name: Date, dtype: int64

Perform data transformation

In [7]:
# Convert confirmed cases and fatalities columns to int as they are counts
train_df['ConfirmedCases'] = train_df['ConfirmedCases'].astype(int)
train_df['Fatalities'] = train_df['Fatalities'].astype(int)

In [8]:
# Convert string date time to datetime object
train_df['Modified_Date'] = pd.to_datetime(train_df['Date'])

In [9]:
# Extract month from datetime object
train_df["month"] = train_df['Modified_Date'].map(lambda x: x.month)

In [10]:
train_df

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,Modified_Date,month
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,2020-01-22,1
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0,0,2020-01-23,1
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0,0,2020-01-24,1
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0,0,2020-01-25,1
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0,0,2020-01-26,1
...,...,...,...,...,...,...,...,...,...,...
16751,26374,NaN,Zambia,-15.4167,28.2833,2020-03-16,0,0,2020-03-16,3
16752,26375,NaN,Zambia,-15.4167,28.2833,2020-03-17,0,0,2020-03-17,3
16753,26376,NaN,Zambia,-15.4167,28.2833,2020-03-18,2,0,2020-03-18,3
16754,26377,NaN,Zambia,-15.4167,28.2833,2020-03-19,2,0,2020-03-19,3


In [11]:
train_df.isnull().sum()

Id                   0
Province/State    9086
Country/Region       0
Lat                 59
Long                59
Date                 0
ConfirmedCases       0
Fatalities           0
Modified_Date        0
month                0
dtype: int64

In [12]:
import ssl 

# Disable SSL certificate verification
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [13]:
# Concatenate latitude and longitude feature together
train_df["geom"] = train_df["Lat"].map(str) + ',' + train_df["Long"].map(str)

In [14]:
# Get new state for dataframe
train_df['new_state'] = train_df['Province/State']

In [15]:
# Get dataframe with null states
null_state_df = train_df[train_df['Province/State'].isnull()==True]
# get all the unique geom from null province dataframe
unique_geom = null_state_df['geom'].unique()

In [16]:
#Get the not null values for province/state
not_null_state_df = train_df[train_df['Province/State'].isnull()==False]
print("Shape of the not null states dataframe",not_null_state_df.shape)
print("Shape of the original dataframe",train_df.shape)
print("Null values of the not null states dataframe",not_null_state_df.isnull().sum())

Shape of the not null states dataframe (7670, 12)
Shape of the original dataframe (16756, 12)
Null values of the not null states dataframe Id                0
Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
ConfirmedCases    0
Fatalities        0
Modified_Date     0
month             0
geom              0
new_state         0
dtype: int64


In [17]:
not_null_state_df["new_state"]  

472      Australian Capital Territory
473      Australian Capital Territory
474      Australian Capital Territory
475      Australian Capital Territory
476      Australian Capital Territory
                     ...             
16456                  United Kingdom
16457                  United Kingdom
16458                  United Kingdom
16459                  United Kingdom
16460                  United Kingdom
Name: new_state, Length: 7670, dtype: object

In [18]:
train_df['geom'].nunique()

276

In [19]:
len(unique_geom)

153

In [20]:
# Get state from the given latitude and longitude
def get_state(coordinates):
    """
    Get states from the given coordinates.
    If the location obtained from the service is None or
    if the inputs are nan's then output is None
    
    Input Args:
    coordinates(str): String with format '<lat>,<long>'
    
    Output Args:
    state(str): String value consisting of state name or Nones
    """
    state = None
    if coordinates == 'nan,nan':
        return state
    location = locator.reverse(coordinates,language='en')
    # If the location is found from the geoenconder
    if location[0] is not None:
        state = location.raw['address'].get('state','')
    return state

In [21]:
# create dictionary of geom to new state using geoencoder
geom_to_state_dict = {}
for i,geom in enumerate(unique_geom):
    state = get_state(geom)
    # If state returned is empty string then default it to None
    if state == "":
        state = None
    geom_to_state_dict[geom] = state

In [22]:
geom_to_state_dict

{'33.0,65.0': 'Helmand',
 '41.1533,20.1683': 'Central Albania',
 '28.0339,1.6596': 'Adrar',
 '42.5063,1.5218': None,
 '17.0608,-61.7964': None,
 '-38.4161,-63.6167': 'La Pampa',
 '40.0691,45.0382': 'Ararat Province',
 'nan,nan': None,
 '47.5162,14.5501': 'Styria',
 '40.1431,47.5769': 'Aghjabadi District',
 '26.0275,50.55': 'Southern Governorate',
 '23.685,90.3563': 'Dhaka Division',
 '13.1939,-59.5432': None,
 '53.7098,27.9534': 'Minsk Region',
 '50.8333,4.0': 'Flanders',
 '9.3077,2.3158': 'Borgou',
 '27.5142,90.4336': 'Trongsa District',
 '-16.2902,-63.5887': 'Santa Cruz',
 '43.9159,17.6791': 'Federation of Bosnia and Herzegovina',
 '-14.235,-51.9253': 'Mato Grosso',
 '4.5353,114.7277': 'Tutong District',
 '42.7339,25.4858': 'Stara Zagora',
 '12.2383,-1.5616': None,
 '11.55,104.9167': 'Phnom Penh',
 '3.8480000000000003,11.5021': 'Centre',
 '6.6111,20.9394': 'Ouaka',
 '-35.6751,-71.543': 'Maule Region',
 '4.5709,-74.2973': 'Cundinamarca',
 '-4.0383,21.7587': 'Kasai',
 '9.7489,-83.7534'

In [23]:
null_state_df['new_state'] = null_state_df['geom'].apply(lambda x:geom_to_state_dict[x])

/home/zoro/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
# Apply tansformations from dictionary to geom column
null_state_df

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,Modified_Date,month,geom,new_state
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,2020-01-22,1,"33.0,65.0",Helmand
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0,0,2020-01-23,1,"33.0,65.0",Helmand
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0,0,2020-01-24,1,"33.0,65.0",Helmand
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0,0,2020-01-25,1,"33.0,65.0",Helmand
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0,0,2020-01-26,1,"33.0,65.0",Helmand
...,...,...,...,...,...,...,...,...,...,...,...,...
16751,26374,NaN,Zambia,-15.4167,28.2833,2020-03-16,0,0,2020-03-16,3,"-15.4167,28.2833",Lusaka Province
16752,26375,NaN,Zambia,-15.4167,28.2833,2020-03-17,0,0,2020-03-17,3,"-15.4167,28.2833",Lusaka Province
16753,26376,NaN,Zambia,-15.4167,28.2833,2020-03-18,2,0,2020-03-18,3,"-15.4167,28.2833",Lusaka Province
16754,26377,NaN,Zambia,-15.4167,28.2833,2020-03-19,2,0,2020-03-19,3,"-15.4167,28.2833",Lusaka Province


In [25]:
null_state_df.isnull().sum()

Id                   0
Province/State    9086
Country/Region       0
Lat                 59
Long                59
Date                 0
ConfirmedCases       0
Fatalities           0
Modified_Date        0
month                0
geom                 0
new_state         1770
dtype: int64

In [26]:
null_state_df['new_state'].unique()

array(['Helmand', 'Central Albania', 'Adrar', None, 'La Pampa',
       'Ararat Province', 'Styria', 'Aghjabadi District',
       'Southern Governorate', 'Dhaka Division', 'Minsk Region',
       'Flanders', 'Borgou', 'Trongsa District', 'Santa Cruz',
       'Federation of Bosnia and Herzegovina', 'Mato Grosso',
       'Tutong District', 'Stara Zagora', 'Phnom Penh', 'Centre', 'Ouaka',
       'Maule Region', 'Cundinamarca', 'Kasai', 'Cartago Province',
       'Vallée du Bandama', 'Villa Clara', 'Cyprus', 'Southeast',
       'Tadjourah', 'Monte Plata', 'Morona Santiago',
       'New Valley Governorate', 'Litoral', 'Manzini', 'Oromia Region',
       'Mainland Finland', 'French Guiana', 'Ogooué-Lolo', 'Lower River',
       'Imereti', 'Hesse', 'Bono East Region',
       'Thessaly - Central Greece', 'Guadeloupe', 'Guam', 'Alta Verapaz',
       'Guernsey', 'Kankan', 'Upper Demerara-Berbice', 'Olancho',
       'Maharashtra', 'Central Kalimantan', 'Isfahan Province',
       'Al Anbar', 'South Di

In [27]:
# Merge the null and not null dataframe
train_df_2 = null_state_df.append(not_null_state_df)
print("Shape of the appended dataframe",train_df_2.shape)
print("Null values of the appended dataframe",train_df_2.isnull().sum())

Shape of the appended dataframe (16756, 12)
Null values of the appended dataframe Id                   0
Province/State    9086
Country/Region       0
Lat                 59
Long                59
Date                 0
ConfirmedCases       0
Fatalities           0
Modified_Date        0
month                0
geom                 0
new_state         1770
dtype: int64


In [28]:
# Remove the rows with null new_state columns
train_df_3 = train_df_2[train_df_2['new_state'].isnull()==False]

In [29]:
# Sort the rows based on ids
train_df_3 = train_df_3.sort_values(by='Id')

In [30]:
train_df_3.isnull().sum()

Id                   0
Province/State    7316
Country/Region       0
Lat                  0
Long                 0
Date                 0
ConfirmedCases       0
Fatalities           0
Modified_Date        0
month                0
geom                 0
new_state            0
dtype: int64

In [31]:
unique_country = train_df_3['Country/Region'].unique()
unique_state = train_df_3['new_state'].unique()

In [32]:
# Create a dictionary to tag string columns to numbers for countries column
unique_country_to_index_dict = {}
index_to_unique_country_dict = {}
for i, val in enumerate(unique_country):
    unique_country_to_index_dict[val] = i
    index_to_unique_country_dict[i] = val

In [33]:
# Create a dictionary to tag string columns to numbers for state column
unique_state_to_index_dict = {}
index_to_unique_state_dict = {}
for i, val in enumerate(unique_state):
    unique_state_to_index_dict[val] = i
    index_to_unique_state_dict[i] = val

In [34]:
# Apply the transformations from dictionary to columns
train_df_3['country_index'] = train_df_3['Country/Region'].apply(lambda x:unique_country_to_index_dict[x])
train_df_3['state_index'] = train_df_3['new_state'].apply(lambda x:unique_state_to_index_dict[x])


In [35]:
def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    # Modifying this part and removing the scaling factor
    # d = 2 * r * asin(sqrt(a)) 
    d = asin(sqrt(a)) 

    return d

In [36]:
# Convert lat long feature to a single feature and fill nan values in the subsequent column
#train_df['lat-long']= [str(x) + '' + str(y) for x, y in zip(train_df['Lat'], train_df['Long'])]
train_df_3['harvesine-lat-long'] = [single_pt_haversine(x, y) for x, y in zip(train_df_3['Lat'], train_df_3['Long'])]

In [37]:
train_df_3

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,Modified_Date,month,geom,new_state,country_index,state_index,harvesine-lat-long
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,2020-01-22,1,"33.0,65.0",Helmand,0,0,0.604242
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0,0,2020-01-23,1,"33.0,65.0",Helmand,0,0,0.604242
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0,0,2020-01-24,1,"33.0,65.0",Helmand,0,0,0.604242
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0,0,2020-01-25,1,"33.0,65.0",Helmand,0,0,0.604242
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0,0,2020-01-26,1,"33.0,65.0",Helmand,0,0,0.604242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16751,26374,NaN,Zambia,-15.4167,28.2833,2020-03-16,0,0,2020-03-16,3,"-15.4167,28.2833",Lusaka Province,132,245,0.278420
16752,26375,NaN,Zambia,-15.4167,28.2833,2020-03-17,0,0,2020-03-17,3,"-15.4167,28.2833",Lusaka Province,132,245,0.278420
16753,26376,NaN,Zambia,-15.4167,28.2833,2020-03-18,2,0,2020-03-18,3,"-15.4167,28.2833",Lusaka Province,132,245,0.278420
16754,26377,NaN,Zambia,-15.4167,28.2833,2020-03-19,2,0,2020-03-19,3,"-15.4167,28.2833",Lusaka Province,132,245,0.278420


In [38]:
train_df_3['harvesine-lat-long'].unique()

array([0.60424194, 0.39292762, 0.24503553, 0.60758611, 0.49972456,
       1.17180616, 1.12025686, 1.1928622 , 1.1317436 , 1.24028837,
       1.11658992, 1.12049264, 1.13711883, 0.97495782, 0.42915248,
       0.51449234, 0.48155829, 0.78824556, 0.51031987, 0.44459561,
       0.08367982, 0.78875406, 0.56483729, 0.40717739, 0.46505044,
       1.00046728, 0.42296247, 0.91287568, 0.10577109, 0.91866639,
       0.96762271, 1.00617468, 0.83075045, 0.64632686, 0.622047  ,
       0.6254123 , 0.75987187, 0.62886577, 0.69964474, 0.87115159,
       0.19123107, 0.65534665, 0.98502144, 0.9587866 , 0.91983788,
       1.00294758, 0.86145995, 0.97221641, 0.9349232 , 0.9164256 ,
       0.94776458, 0.958643  , 0.99709368, 0.95490159, 0.97989895,
       0.95086961, 0.95236728, 0.93328925, 0.99802   , 0.98276864,
       1.00606813, 0.99382216, 0.97491265, 0.89710548, 0.82783696,
       0.91914288, 0.98030243, 1.01669411, 0.93807445, 0.8806474 ,
       0.96694616, 0.77123436, 0.75411841, 0.87615845, 1.01195

In [39]:
train_df_3[train_df_3['harvesine-lat-long'].isnull()==True]

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities,Modified_Date,month,geom,new_state,country_index,state_index,harvesine-lat-long


In [40]:
# Extract data based on months
month1_df = train_df_3[train_df_3['month'] == 1]
month2_df = train_df_3[train_df_3['month'] == 2]
month3_df = train_df_3[train_df_3['month'] == 3]

In [41]:
train_df_3.columns

Index(['Id', 'Province/State', 'Country/Region', 'Lat', 'Long', 'Date',
       'ConfirmedCases', 'Fatalities', 'Modified_Date', 'month', 'geom',
       'new_state', 'country_index', 'state_index', 'harvesine-lat-long'],
      dtype='object')

In [46]:
# Select columns which will be used for model training
selected_columns = ['Id', 'Modified_Date', 'month', 'country_index', 
                    'state_index', 'harvesine-lat-long', 
                    'ConfirmedCases', 'Fatalities']

In [47]:
# Create final training dataframe
final_train_df = train_df_3[selected_columns]

In [48]:
# Check if save path exists, if not then create one
save_path = current_path + '/Data'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

In [49]:
# Save dataframe as csv in the save path
final_train_df.to_csv(save_path+'/train_model1.csv')